In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="9e5f390cdd7e44658f17b1b3228c02b2",
                                                           client_secret="a2d7e42715d54e12a3a4e86ba5d93dc0"))

In [ ]:
"""
Primary:
- id (Id of track generated by Spotify)
Numerical:
- acousticness (Ranges from 0 to 1)
- danceability (Ranges from 0 to 1)
- energy (Ranges from 0 to 1)
- duration_ms (Integer typically ranging from 200k to 300k)
- instrumentalness (Ranges from 0 to 1)
- valence (Ranges from 0 to 1)
- popularity (Ranges from 0 to 100)
- tempo (Float typically ranging from 50 to 150)
- liveness (Ranges from 0 to 1)
- loudness (Float typically ranging from -60 to 0)
- speechiness (Ranges from 0 to 1)
- year (Ranges from 1921 to 2020)
Dummy:
- mode (0 = Minor, 1 = Major)
- explicit (0 = No explicit content, 1 = Explicit content)
Categorical:
- key (All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1 and so on…)
- artists (List of artists mentioned)
- release_date (Date of release mostly in yyyy-mm-dd format, however precision of date may vary)
- name (Name of the song)"""

In [2]:
# reading csv
df = pd.read_csv('suggestor/edited_data_v2.csv')

In [3]:
df

,Unnamed: 0,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,combined
0,0,0.991000,Mamie Smith,0.598,168333,0.2240,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,0.0936,149.976,0.6340,1920,Keep A Song In Your Soul - Mamie Smith
1,1,0.643000,Screamin Jay Hawkins,0.852,150200,0.5170,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,0.0534,86.889,0.9500,1920,I Put A Spell On You - Screamin Jay Hawkins
2,2,0.993000,Mamie Smith,0.647,163827,0.1860,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,0.1740,97.600,0.6890,1920,Golfing Papa - Mamie Smith
3,3,0.000173,Oscar Velazquez,0.730,422087,0.7980,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,0.0425,127.997,0.0422,1920,True House Music - Xavier Santos & Carlos Gomi...
4,4,0.295000,Mixe,0.704,165224,0.7070,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,0.0768,122.076,0.2990,1920,Xuniverxe - Mixe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159436,174365,0.976000,"Ludovico Einaudi, Johannes Bornlöf",0.358,381150,0.1210,0,0sZ6HFSulsrOl6VTBwp6jd,0.889000,2,0.1350,-25.111,1,I giorni,0,0.0532,131.808,0.1060,2021,"I giorni - Ludovico Einaudi, Johannes Bornlöf"
159437,174369,0.995000,"Ludovico Einaudi, Johannes Bornlöf",0.297,349200,0.0287,0,2LeqqwzobL5ktfQhWA3bHh,0.908000,8,0.0995,-30.008,1,Nuvole bianche,0,0.0564,141.636,0.0678,2021,"Nuvole bianche - Ludovico Einaudi, Johannes Bo..."
159438,174371,0.995000,"Ludovico Einaudi, Johannes Bornlöf",0.343,206700,0.0165,0,3Glmyv3hbGGTgeR3FZrxJA,0.878000,9,0.0774,-30.915,0,Una Mattina,0,0.0455,126.970,0.1510,2021,"Una Mattina - Ludovico Einaudi, Johannes Bornlöf"
159439,174375,0.988000,"Ludovico Einaudi, Johannes Bornlöf",0.316,303333,0.0573,0,6QGVWUbmlePAiY5zJjfCmT,0.879000,3,0.1200,-24.121,1,Night,0,0.0515,81.070,0.0373,2021,"Night - Ludovico Einaudi, Johannes Bornlöf"


In [4]:
# adding direct url to data set by adding url prefix and id
url = 'http://open.spotify.com/track/' + df['id']
df['url'] = url

In [5]:
# reordering columns, leaving out ID and release date
df = df[['combined', 'url', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo', 'valence']]

In [6]:
df

,combined,url,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,Keep A Song In Your Soul - Mamie Smith,http://open.spotify.com/track/0cS0A1fUEUd1EW3F...,1920,0.991000,0.598,168333,0.2240,0,0.000522,5,0.3790,-12.628,0,12,0.0936,149.976,0.6340
1,I Put A Spell On You - Screamin Jay Hawkins,http://open.spotify.com/track/0hbkKFIJm7Z05H8Z...,1920,0.643000,0.852,150200,0.5170,0,0.026400,5,0.0809,-7.261,0,7,0.0534,86.889,0.9500
2,Golfing Papa - Mamie Smith,http://open.spotify.com/track/11m7laMUgmOKqI3o...,1920,0.993000,0.647,163827,0.1860,0,0.000018,0,0.5190,-12.098,1,4,0.1740,97.600,0.6890
3,True House Music - Xavier Santos & Carlos Gomi...,http://open.spotify.com/track/19Lc5SfJJ5O1oaxY...,1920,0.000173,0.730,422087,0.7980,0,0.801000,2,0.1280,-7.311,1,17,0.0425,127.997,0.0422
4,Xuniverxe - Mixe,http://open.spotify.com/track/2hJjbsLCytGsnAHf...,1920,0.295000,0.704,165224,0.7070,1,0.000246,10,0.4020,-6.036,0,2,0.0768,122.076,0.2990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159436,"I giorni - Ludovico Einaudi, Johannes Bornlöf",http://open.spotify.com/track/0sZ6HFSulsrOl6VT...,2021,0.976000,0.358,381150,0.1210,0,0.889000,2,0.1350,-25.111,1,0,0.0532,131.808,0.1060
159437,"Nuvole bianche - Ludovico Einaudi, Johannes Bo...",http://open.spotify.com/track/2LeqqwzobL5ktfQh...,2021,0.995000,0.297,349200,0.0287,0,0.908000,8,0.0995,-30.008,1,0,0.0564,141.636,0.0678
159438,"Una Mattina - Ludovico Einaudi, Johannes Bornlöf",http://open.spotify.com/track/3Glmyv3hbGGTgeR3...,2021,0.995000,0.343,206700,0.0165,0,0.878000,9,0.0774,-30.915,0,0,0.0455,126.970,0.1510
159439,"Night - Ludovico Einaudi, Johannes Bornlöf",http://open.spotify.com/track/6QGVWUbmlePAiY5z...,2021,0.988000,0.316,303333,0.0573,0,0.879000,3,0.1200,-24.121,1,0,0.0515,81.070,0.0373


In [47]:
# finding a song I want to use as an input
tay = df[df['combined'] == 'Heather - CONAN GRAY'].sort_values('popularity', ascending=False)
tay

,combined,url,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
18676,Heather - CONAN GRAY,http://open.spotify.com/track/4xqrdfXkTW4T0Rau...,2020,0.584,0.357,198040,0.425,0,0.0,5,0.322,-7.301,1,91,0.0333,102.078,0.27


In [7]:
# target set will be both artist and name
y_set = ['combined', 'url']

# droping target from data matrix
df_data = df.drop(y_set, axis=1)

# set target
df_target = df[y_set]

In [8]:
# fit on data, 12 neighbors
nn = NearestNeighbors(algorithm='brute', metric='cosine', leaf_size =15, n_neighbors=12, n_jobs=-1)
nn.fit(df_data)

NearestNeighbors(algorithm='brute', leaf_size=15, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=12, p=2,
                 radius=1.0)

In [9]:
# sample a song(index) from df_data to use as our query point 
input_index = 18676 # 'Heather - CONAN GRAY

# vectorize 
data_vect = [df_data.iloc[input_index].values]
data_vect

[array([ 2.02000e+03,  5.84000e-01,  3.57000e-01,  1.98040e+05,
         4.25000e-01,  0.00000e+00,  0.00000e+00,  5.00000e+00,
         3.22000e-01, -7.30100e+00,  1.00000e+00,  9.10000e+01,
         3.33000e-02,  1.02078e+02,  2.70000e-01])]

In [10]:
# Query Using kneighbors 
neigh_dist, neigh_indices = nn.kneighbors(data_vect)

In [11]:
# top 12 closest data vectors to our reference vector, data_vect
neigh_dist

array([[1.11022302e-16, 1.17614551e-09, 2.32895814e-09, 2.42881959e-09,
        2.44376641e-09, 3.14857507e-09, 3.19905580e-09, 3.64807407e-09,
        3.79034759e-09, 4.24102120e-09, 4.36705216e-09, 4.47346948e-09]])

In [12]:
# these are the corresponding indicies of the most similar vectors 
neigh_indices

array([[ 18676,  71444,  36663,  18392,  17769,  18753,  36434,  54116,
         17774,  15379, 101855, 131535]], dtype=int64)

In [13]:
# breaking down the array to a list
indexs = neigh_indices.flat[0:12].tolist()
indexs

[18676,
 71444,
 36663,
 18392,
 17769,
 18753,
 36434,
 54116,
 17774,
 15379,
 101855,
 131535]

In [14]:
# result metrics
df_data.iloc[indexs]

,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
18676,2020,0.5840,0.357,198040,0.425,0,0.000000,5,0.3220,-7.301,1,91,0.0333,102.078,0.270
71444,2020,0.8190,0.854,197587,0.525,0,0.000000,1,0.1460,-4.426,1,86,0.2140,97.054,0.630
36663,2019,0.1240,0.640,197867,0.648,0,0.000000,5,0.0802,-8.113,0,81,0.1870,92.977,0.548
18392,2018,0.5350,0.752,197437,0.505,0,0.000003,9,0.1040,-7.621,1,83,0.2530,91.970,0.419
17769,2015,0.0371,0.764,196520,0.705,0,0.000019,3,0.0943,-5.279,0,82,0.0278,101.003,0.672
18753,2020,0.0112,0.746,199054,0.765,0,0.000000,6,0.0936,-4.410,0,91,0.0993,114.044,0.737
36434,2018,0.0558,0.575,197947,0.571,0,0.000000,1,0.0912,-7.906,1,76,0.0439,105.049,0.449
54116,2018,0.0211,0.568,198160,0.656,1,0.000014,8,0.2130,-5.413,1,75,0.0433,100.049,0.364
17774,2015,0.2230,0.765,198293,0.356,1,0.000000,11,0.0963,-5.556,0,78,0.1950,96.991,0.189
15379,2001,0.2360,0.793,196160,0.832,0,0.000011,1,0.2020,-4.862,0,74,0.0406,107.661,0.872


In [15]:
# result target
df_target.iloc[indexs]

,combined,url
18676,Heather - Conan Gray,http://open.spotify.com/track/4xqrdfXkTW4T0Rau...
71444,Mi Cuarto - Jerry Di,http://open.spotify.com/track/6fmQjRuo7XvRFCK2...
36663,"Beautiful People (feat. Khalid) - Ed Sheeran, ...",http://open.spotify.com/track/4evmHXcjt3bTUHD1...
18392,I Like Me Better - Lauv,http://open.spotify.com/track/2P91MQbaiQOfbiz9...
17769,Wait a Minute! - WILLOW,http://open.spotify.com/track/0y60itmpH0aPKsFi...
18753,Dynamite - BTS,http://open.spotify.com/track/4saklk6nie3yiGeP...
36434,"This Feeling - The Chainsmokers, Kelsea Ballerini",http://open.spotify.com/track/4NBTZtAt1F13VvlS...
54116,breathin - Ariana Grande,http://open.spotify.com/track/4OafepJy2teCjYJb...
17774,Don't - Bryson Tiller,http://open.spotify.com/track/3pXF1nA74528Edde...
15379,"Whenever, Wherever - Shakira",http://open.spotify.com/track/2lnzGkdtDj5mtlcO...
